## Tbrquiv ##

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import raytrace as rt
import numpy as np
import math
from pylab import *
from raytrace import implane
import rtcore

/home/hp/miniconda3/envs/py27/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
# Plasma frequency (Hz) → Electron density (cm^-3)
def pf2ne(pf_hz):
    return (pf_hz/9000)**2

# Electron density (cm^-3) → Plasma frequency (Hz)
def ne2pf(ne):
    return 9000*(ne)**0.5

In [3]:
# Global constants & Derived Quantities
DeltaS=0.01  # Initial ray path increment in solar radii
freq_hz = 1000.e6 # Hz, radio wave frequency
c = 3.e10 # in cm/s
Rsun  = 7.e9 # in cm
Rsun_km = Rsun / 1.e5
omega = 2*np.pi*freq_hz # rad/s, radio wave frequency
freq2 = freq_hz * freq_hz # Hz^2, radio wave frequency squared
omega2 = omega * omega # (rad/s)^2, radio wave frequency squared
k0 = omega / c # # k0 = omega/c_light_cms, wave number in a vacuum
k0_Rsun = k0 * Rsun # k0 = omega/c_light_Rsun, wave # in rad/Rsun
rhocr =  pf2ne(freq_hz) # Critical density at given self.freq
rhocr_inv = 1 / rhocr # 1/self.RhoCr
tol = 0.005
tol2 = tol**2
#ds = np.empty((ny, nx), dtype=np.double)
#ds[:] = self.DeltaS
#absminstep = 1e-4*np.average(ds)Coulomb_log
absminstep = 1e-4
toleps = 1e-6
cntmax = 50.
h_chromo_km = 10000
r_chromo = (Rsun_km + h_chromo_km - 1000.)/Rsun_km
r_chro_cor = (Rsun_km + h_chromo_km)/Rsun_km
r_corona = (Rsun_km + h_chromo_km + 1000.)/Rsun_km
c_light_cms = 2.9979245800e10
ProtonChargeCGSe = 4.80320427e-10
ProtonMass_g = 1.672621638e-24
ProtonMassInv = 1./ProtonMass_g
ElectronMass_g = 9.10938215e-28
Boltzmannk_ergK = 1.380650424e-16
Rsun_cm = 6.955e10
Rsun_km = 6.955e5
Te_corona_K = 1.0e6
Te_chromo_K = 3.0e4
AU_m = 149597870700
AU_Rsun = 215.097
Coulomb_log = 20.0
Msun_G = 1.0
Mdir = np.array((0., 0., 1.), dtype=np.double)
e2 = (ProtonChargeCGSe)**2
e2w2 = e2/omega2
e_ovr_mcw2 = e2w2/((ElectronMass_g*                  \
                           c_light_cms)**2)
e2_4pi_ovr_m = 4.*np.pi*e2/ElectronMass_g
e2_4pi_ovr_mw2 = 4.*np.pi*e2w2/ElectronMass_g
twokf2c2 = 2.*Boltzmannk_ergK*freq2/            \
                   c_light_cms
lnLambda_13p7 = 13.7*Coulomb_log
cnu = 5.5
callcount = 1.0


## Simulation code starts from here ##

In [4]:

# Initialize image plane
implane = rt.implane([60,60],
                   [-2.0,-2.0,2.0,2.0],
                   (215,0,0),
                   25.0,
                   freq=freq_hz,
                   mode='TbrIQUV',
                   trknpmax=None)
implane.package = '/home/hp/raytrace/py_raytr_threaded'
#trace.set_plfunc('/home/hp/raytrace_scripts/plasma_parameters.c')
#rtcore.remove_streamers()
#rtcore.make_streamer(90,90,0)
implane.arprm = np.array((
            DeltaS,     
            #self.rsph+0.01,  # Radius of the integrarion sphere + a bit more
            freq_hz,       
            omega,      
            freq2,      
            omega2,     
            k0,         
            k0_Rsun,    
            rhocr,           
            rhocr_inv,  
            tol,        
            tol2,       
            absminstep, 
            toleps,     
            cntmax,      # Upper limit to Newton iterations number
            h_chromo_km,     # km, Chromosphere thickness
            r_chromo,   # in Rsun units: "top" of chromosphere
            r_chro_cor, # in Rsun units: chromosphere-corona "border"
            r_corona,   # in Rsun units: "bottom" of corona
            c_light_cms,      # cm/s, Speed of light
            ProtonChargeCGSe,   # StatCoulombs, CGSe
            ProtonMass_g,       # g
            ProtonMassInv,      # 1/g
            ElectronMass_g,     # g
            Boltzmannk_ergK,    # erg/K, Boltzmann constant 
            Rsun_cm,         # cm, Solar radius
            Rsun_km,         # km, Solar radius
            Te_corona_K,     # K
            Te_chromo_K,     # K
            AU_m,      # Sun-earth distance (1 AU) in meters
            AU_Rsun,   # Sun-earth distance (1 AU) in solar radii
            Msun_G,    # G/Rsun^3, solar dipole field at equator
            Mdir[0],         # Solar dipole x direction, CGI
            Mdir[1],         # Solar dipole y direction, CGI
            Mdir[2],         # Solar dipole z direction, CGI
            e_ovr_mcw2,           # (e/mcw)^2
            e2_4pi_ovr_m,
            e2_4pi_ovr_mw2,
            twokf2c2,
            lnLambda_13p7,
            cnu,             # Coef. at Ginzburg's nu_eff
            callcount        # Number of calls to advance_beam()
            ), dtype=np.double)
                    
print(implane.arprm)

plf_cname =  plasma_parameters.c
fname =  plasma_parameters.c
dname =  /home/hp/raytrace_work/raytrace_scripts
bfname =  plasma_parameters.c
name =  plasma_parameters
gcc -g -fPIC  -I/home/hp/miniconda3/envs/py27/lib/python2.7/site-packages/raytrace/inc -c /home/hp/raytrace_work/raytrace_scripts/streamer.c -o streamer.o
gcc -g -fPIC  -I/home/hp/miniconda3/envs/py27/lib/python2.7/site-packages/raytrace/inc -c plasma_parameters.c -o plasma_parameters.o
gcc -shared streamer.o plasma_parameters.o -L/home/hp/miniconda3/envs/py27/lib/python2.7/site-packages/raytrace/lib -L/usr/lib -lm -lmxv -o plasma_parameters.so
[1.00000000e-02 1.00000000e+09 6.28318531e+09 1.00000000e+18
 3.94784176e+19 2.09439510e-01 1.46607657e+09 1.23456790e+10
 8.10000000e-11 5.00000000e-03 2.50000000e-05 1.00000000e-04
 1.00000000e-06 5.00000000e+01 1.00000000e+04 1.12857143e+00
 1.14285714e+00 1.15714286e+00 2.99792458e+10 4.80320427e-10
 1.67262164e-24 5.97863843e+23 9.10938215e-28 1.38065042e-16
 6.95500000e+10 6.

In [ ]:
# Ray-tracing configuration
trkparms_=['bfield', 'arclen', 'rho', 'Stokes','TbrIQUV']
trkrays_=np.array([[2,5],[3,3],[4,4]])

# Run ray tracing
implane.trace(200,trkrays=trkrays_, trkparms=trkparms_)
print(implane.traj)

None


In [9]:
# Output directory
BASE_OUTDIR = "raytrace_results"
if not os.path.exists(BASE_OUTDIR):
    os.makedirs(BASE_OUTDIR)

# Create output folder
freq_tag = "freq_%dMHz" % int(freq_hz / 1e6)
outdir = os.path.join(BASE_OUTDIR, freq_tag)
if not os.path.exists(outdir):
    os.makedirs(outdir)

# Save numerical output
tbriquv = trace.tbriquv
tbriquv_traj = trace.traj.tbriquv
np.save(os.path.join(outdir, "tbriquv.npy"), tbriquv)

AttributeError: 'NoneType' object has no attribute 'tbriquv'

In [ ]:
# Save quick-look image
plt.figure(figsize=(6, 5))
plt.imshow(tbriquv[:, :, 0], cmap='gist_heat')  # Stokes-I
plt.colorbar(label="Brightness Temperature")
plt.title("Stokes-I @ {0:.1f} MHz".format(freq_hz/1e6))
plt.tight_layout()
plt.savefig(os.path.join(outdir, "tbriquv.png"), dpi=200)
plt.show()

In [ ]:
# Save metadata
with open(os.path.join(outdir, "params.txt"), "w") as f:
    f.write("Frequency (Hz): %g\n" % freq_hz)
    f.write("Frequency (MHz): %.2f\n" % (freq_hz/1e6))
    f.write("Critical density (cm^-3): %.3e\n" % rhocr)
    f.write("Ray indices: %s\n" % trkrays_.tolist())
    f.write("Tracked parameters: %s\n" % trkparms_)

In [ ]:
print("Saved results in:", outdir)
print("\n All simulations completed.")